In [ ]:
import os
import shutil

def process_data_folder(data_path, folder_name):
    
    folder_path = os.path.join(data_path, folder_name)
    output_path = os.path.join("/kaggle/working", folder_name)
    images_output = os.path.join(output_path, 'images')
    labels_output = os.path.join(output_path, 'labels')
    os.makedirs(images_output, exist_ok=True)
    os.makedirs(labels_output, exist_ok=True)

    images_folder = os.path.join(folder_path, 'images')
    detections_folder = os.path.join(folder_path, 'detections')

    for subject in os.listdir(images_folder):
        subject_images_path = os.path.join(images_folder, subject)
        subject_detections_path = os.path.join(detections_folder, subject)

        if not os.path.isdir(subject_images_path):
            continue

        for image_name in os.listdir(subject_images_path):
            image_path = os.path.join(subject_images_path, image_name)
            image_id = os.path.splitext(image_name)[0]  

            new_image_name = f"{subject}_{image_id}.png"
            new_image_path = os.path.join(images_output, new_image_name)

            shutil.copy(image_path, new_image_path)

            label_file = os.path.join(subject_detections_path, f"{image_id}.txt")  
            new_label_name = f"{subject}_{image_id}.txt"  
            new_label_path = os.path.join(labels_output, new_label_name)

            if os.path.exists(label_file):
                with open(label_file, 'r') as f:
                    labels = f.readlines()

                if labels:
                    with open(new_label_path, 'w') as f:
                        f.writelines(labels)
                else:
                    print(f"Warning: The label file for {image_name} is empty. Creating empty label.")
                    open(new_label_path, 'w').close()
            else:
                open(new_label_path, 'w').close()
                print(f"Warning: No label file found for {image_name}. Creating empty label.")

    print(f"Processing of {folder_name} complete. Images saved in {images_output}, labels saved in {labels_output}.")

data_path = "/kaggle/input/computer-vision-project"
process_data_folder(data_path, 'train')
process_data_folder(data_path, 'val')

In [ ]:
file_path = '/kaggle/working/train/labels/Subject_0_304.txt'

with open(file_path, 'r') as file:
    content = file.read()

print(content)

In [ ]:
file_path = '/kaggle/working/train/labels/Subject_1_511.txt'

with open(file_path, 'r') as file:
    content = file.read()

print(content)

In [ ]:
import os
from PIL import Image

def normalize_coordinates(image_width, image_height, xmin, ymin, xmax, ymax):
    
    x_center = (xmin + xmax) / 2 / image_width
    y_center = (ymin + ymax) / 2 / image_height
    width = (xmax - xmin) / image_width
    height = (ymax - ymin) / image_height
    return f"1 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

def process_labels(labels_folder, images_folder):
   
    for label_file in os.listdir(labels_folder):
        
        label_path = os.path.join(labels_folder, label_file)
        image_name = os.path.splitext(label_file)[0] + ".png"  
        image_path = os.path.join(images_folder, image_name)
        
        if not os.path.exists(image_path):
            print(f"Image not found for {label_file}. Skipping.")
            continue
        
        with Image.open(image_path) as img:
            image_width, image_height = img.size
        
        with open(label_path, 'r') as file:
            content = file.read().strip()
        
        if content:  
            yolo_lines = []
            for line in content.splitlines():
                try:
                    xmin, ymin, xmax, ymax = map(int, line.split(','))
                    yolo_lines.append(normalize_coordinates(image_width, image_height, xmin, ymin, xmax, ymax))
                except ValueError:
                    print(f"Invalid content in {label_file}, skipping line: {line}")
            
            yolo_content = "\n".join(yolo_lines)
        else: 
            yolo_content = ""
        
        with open(label_path, 'w') as file:
            file.write(yolo_content)
        
        print(f"Processed and updated: {label_file}")

labels_folder = "/kaggle/working/train/labels"
images_folder = "/kaggle/working/train/images"

process_labels(labels_folder, images_folder)


In [ ]:
image_path = '/kaggle/working/val/images/Subject_58_301.png'
label_path = '/kaggle/working/val/labels/Subject_58_301.txt'

show_bounding_boxes(image_path, label_path)

In [ ]:
image_path = '/kaggle/working/val/images/Subject_60_50.png'
label_path = '/kaggle/working/val/labels/Subject_60_50.txt'

show_bounding_boxes(image_path, label_path)

In [ ]:
pip install Ultralytics

In [ ]:
from ultralytics import YOLO
model= YOLO("yolo11m.pt")

In [ ]:
yaml_content = """
path: /kaggle/working/
train: /kaggle/working/train/images
val: /kaggle/working/val/images
test: 

nc: 2
names:
  0: 'no_tumor'
  1: 'tumor'

"""

yaml_file_path = '/kaggle/working/data.yaml'

with open(yaml_file_path, 'w') as file:
    file.write(yaml_content)

print(f"YAML configuration file saved at: {yaml_file_path}")

In [ ]:
model.train(data='/kaggle/working/data.yaml', epochs=100, imgsz=800, batch=16, patience=20, optimizer="adamW",lr0=0.0001)

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

trained_model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

all_results = trained_model(source="/kaggle/working/val/images" ,conf=0.2 ,save=False)

image_dir = "/kaggle/working/val/images"
image_files = sorted(os.listdir(image_dir))  

for i, result in enumerate(all_results[:]):  
    image_path = os.path.join(image_dir, image_files[i])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  

    annotated_image = result.plot()
    
    plt.figure(figsize=(8, 8))
    plt.imshow(annotated_image)
    plt.axis("off")
    plt.title(image_files[i])  
    plt.show()

In [ ]:
results = model.val()
print(results)